## Calculate Phi vertical mean for a particular layer and LME

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import pandas as pd
import utils
import warnings
warnings.filterwarnings('ignore')
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.pyplot as plt

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'

In [3]:
filename = f'{path}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_monthly.nc'
dset = xr.open_dataset(filename)
dz = dset['dz']/1e2 # m
darea = dset['TAREA']/1e10 #km*km
dvol = dz*darea

In [4]:
KMT = dset['KMT']
nlat = dset.dims['nlat']
nlon = dset.dims['nlon']
for xx in range(nlat):
    for yy in range(nlon):
        kk = np.int(KMT[xx, yy])
        if kk < 60:
            dvol[kk:, xx, yy] = float("nan")

In [5]:
#filename = f'{path}/DPLE-FOSI_hindcast_Phi_1948-2017_monthly.nc'
#filename = f'{path}/DPLE-FOSI_hindcast_Phi_Eo_space_1948-2017_monthly.nc'
filename = f'{path}/DPLE-FOSI_hindcast_Phi_mean_dimension_reduced_1948-2017_monthly.nc'
ds = xr.open_dataset(filename)
dsf = xr.Dataset()
for ii in range(3):
    print(ii)
    Phi_i = ds[f'Phi_{ii}']*dvol
    # select a vertical layer
    dsf[f'Phi_{ii}_0-200m'] = Phi_i.isel(z_t=range(0, 20, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(0, 20, 1)).sum(dim='z_t'))
    dsf[f'Phi_{ii}_200-600m'] = Phi_i.isel(z_t=range(20, 35, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(20, 35, 1)).sum(dim='z_t'))

0
1
2


In [6]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:         (nlat: 384, nlon: 320)
Coordinates:
    TLAT            (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.19 72.19
    TLONG           (nlat, nlon) float64 320.6 321.7 322.8 ... 318.9 319.4 319.8
    ULAT            (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.41 72.41
    ULONG           (nlat, nlon) float64 321.1 322.3 323.4 ... 319.2 319.6 320.0
Dimensions without coordinates: nlat, nlon
Data variables:
    Phi_0_0-200m    (nlat, nlon) float64 nan nan nan nan nan ... nan nan nan nan
    Phi_0_200-600m  (nlat, nlon) float64 nan nan nan nan nan ... nan nan nan nan
    Phi_1_0-200m    (nlat, nlon) float64 nan nan nan nan nan ... nan nan nan nan
    Phi_1_200-600m  (nlat, nlon) float64 nan nan nan nan nan ... nan nan nan nan
    Phi_2_0-200m    (nlat, nlon) float64 nan nan nan nan nan ... nan nan nan nan
    Phi_2_200-600m  (nlat, nlon) float64 nan nan nan nan nan ... nan nan nan nan

In [7]:
%time
#dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_vertical_mean_1948-2017_monthly.nc', mode='w')
#dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_Eo_space_vertical_mean_1948-2017_monthly.nc', mode='w')
dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_mean_dimension_reduced_vertical_mean_1948-2017_monthly.nc', mode='w')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.44 µs


## calculate LME-averaged Phi 

In [3]:
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE-FOSI_hindcast_Phi_mean_dimension_reduced_vertical_mean_1948-2017_monthly.nc'
dsf = xr.open_dataset(filename)

In [4]:
datadir = f'/glade/u/home/{USER}/O2prediction/Prediction/notebooks/data'
filename = '%s/LME-mask-POP_gx1v6.nc' % datadir
ds1 = xr.open_dataset(filename)
lme_mask = ds1['lme_mask']

In [5]:
#lme = range(np.int(lme_mask.min()), np.int(lme_mask.max()), 1)
lme = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 65]

In [8]:
boxPhi = xr.Dataset()
for ii in range(3):
    print(ii)
    phi_box1 = np.ma.zeros(len(lme))
    phi_box1.mask = True
    phi_box2 = phi_box1.copy()
    for box,hh in zip(lme, range(len(lme))):
        latloc, lonloc = np.where(lme_mask == box)
        ar1 = np.ma.zeros([len(latloc)]);ar1.mask=True
        ar2 = ar1.copy()
        for bb in range(len(latloc)):
            ar1[bb] = dsf[f'Phi_{ii}_0-200m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
            ar2[bb] = dsf[f'Phi_{ii}_200-600m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
        phi_box1[hh] = np.nanmean(ar1,axis=0)
        phi_box2[hh] = np.nanmean(ar2,axis=0)
#
    boxPhi[f'Phi_{ii}_0-200m'] = xr.DataArray(data=phi_box1, coords=dict(lme=('lme', lme)), dims=('lme'))
    boxPhi[f'Phi_{ii}_200-600m'] = xr.DataArray(data=phi_box2, coords=dict(lme=('lme', lme)), dims=('lme'))

0
1
2


In [9]:
boxPhi.load()

<xarray.Dataset>
Dimensions:         (lme: 11)
Coordinates:
  * lme             (lme) int64 1 2 3 4 5 6 7 8 9 10 65
Data variables:
    Phi_0_0-200m    (lme) float64 1.171 1.267 1.112 0.6169 ... 1.237 1.014 1.069
    Phi_0_200-600m  (lme) float64 0.1809 0.2588 0.1582 ... 1.185 0.7968 0.1997
    Phi_1_0-200m    (lme) float64 1.483 1.476 1.142 ... 1.571 0.8958 1.311
    Phi_1_200-600m  (lme) float64 0.2238 0.3086 0.1801 ... 1.492 0.8494 0.2449
    Phi_2_0-200m    (lme) float64 2.162 1.889 1.201 ... 2.308 0.7396 1.817
    Phi_2_200-600m  (lme) float64 0.3141 0.4085 0.2218 ... 2.155 0.9466 0.3389

In [10]:
dout = f'/glade/scratch/{USER}/DPLE-results'
#boxPhi.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_LME_1948-2017_monthly.nc', mode='w')
#boxPhi.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_Eo_space_LME_1948-2017_monthly.nc', mode='w')
boxPhi.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_mean_dimension_reduced_LME_1948-2017_monthly.nc', mode='w')